In [67]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
import torch
from torch.utils import data
from torch import nn
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import LinearConstraint, NonlinearConstraint, Bounds, minimize

# Load Data

In [68]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [69]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [70]:
config = initializeConfig()
obs = getObs(400)

In [71]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df.iloc[284]

0         3.734223
1         7.875106
2         2.509849
3         7.372139
4         8.753891
5         7.977354
6         7.539283
7         8.558976
8     32187.640000
9         5.071216
10       99.001530
Name: 284, dtype: float64

In [73]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [74]:
filtered_df = df[df['Time'] > 90]
filtered_df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


# NN Stability Classifier

In [75]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [77]:
from torch.utils import data
def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [78]:
classif = Classifier()
classif.double()

poly = PolynomialFeatures(2)

raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10
idxs = np.array(list(range(0, 300)))# + list(range(0))) # FOR CV

train_X = torch.from_numpy(poly.fit_transform(raw_X.iloc[idxs])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y.iloc[idxs]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[300:400])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[300:400]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [79]:
loss = nn.BCELoss()
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
def train(num_epochs, trainer, loss):
    for epoch in range(num_epochs):

        for X, y in data_iter:
            X.requires_grad = True
            l = loss(classif(X) ,y)

            trainer.zero_grad() #sets gradients to zero

            l.backward() # back propagation

            trainer.step() # parameter update

        l = loss(classif(train_X), train_Y)
        if epoch % 50 == 0:
            print(f'epoch {epoch + 1}, loss {l:f}')
            #print(X.grad)

In [80]:
train(num_epochs, trainer, loss), sum((classif(test_X) >= 0.5) == test_Y)/len(test_Y)

epoch 1, loss 0.542237
epoch 51, loss 1.138355
epoch 101, loss 0.001929
epoch 151, loss 0.000670
epoch 201, loss 0.000374


(None, tensor([0.9100]))

# Altitude Regression

In [81]:
poly = PolynomialFeatures(2)
raw = filtered_df.iloc[:,:8]
qr = LinearRegression()
idxs = np.array(list(range(150)))# + list(range(50)))

ridge = Ridge(alpha=0.6)
ridge.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
lasso = Lasso(alpha=.2)
lasso.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])

qr.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
print('R2', ridge.score(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3]))
print('Test RMSE Quad', np.sqrt(MSE(filtered_df.iloc[150:, -3], qr.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE ridge', np.sqrt(MSE(filtered_df.iloc[150:, -3], ridge.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE lasso', np.sqrt(MSE(filtered_df.iloc[150:, -3], lasso.predict(poly.fit_transform(raw[150:])))))

print('Train RMSE', np.sqrt(MSE(filtered_df.iloc[idxs, -3], ridge.predict(poly.fit_transform(raw.iloc[idxs])))))

R2 0.9929351496740758
Test RMSE Quad 1815.417729777189
Test RMSE ridge 2056.5256021326427
Test RMSE lasso 1815.019990798251
Train RMSE 1186.314188487517


# Solving COP

In [82]:
def stab_constraint(X):
    with torch.no_grad():
        return classif(torch.from_numpy(poly.fit_transform([X])))[0] - 0.6
def l0(X):
    return X[0]-2
def l1(X):
    return X[1]-2
def l2(X):
    return X[2]-2
def l3(X):
    return X[3]-2
def l4(X):
    return X[4]-2
def l5(X):
    return X[5]-2
def l6(X):
    return X[6]-2
def l7(X):
    return X[7]-2

def u0(X):
    return 10-X[0]
def u1(X):
    return 10-X[1]
def u2(X):
    return 10-X[2]
def u3(X):
    return 10-X[3]
def u4(X):
    return 10-X[4]
def u5(X):
    return 10-X[5]
def u6(X):
    return 10-X[6]
def u7(X):
    return 10-X[7]
def objective(X):
    return -ridge.predict(poly.fit_transform([X]))[0]
classif.eval()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [83]:
ineq_cons = {'type': 'ineq',
             'fun' : stab_constraint}
L0 = {'type': 'ineq',
             'fun' : l0}
L1 = {'type': 'ineq',
             'fun' : l1}
L2 = {'type': 'ineq',
             'fun' : l2}
L3 = {'type': 'ineq',
             'fun' : l3}
L4 = {'type': 'ineq',
             'fun' : l4}
L5 = {'type': 'ineq',
             'fun' : l5}
L6 = {'type': 'ineq',
             'fun' : l6}
L7 = {'type': 'ineq',
             'fun' : l7}
U0 = {'type': 'ineq',
             'fun' : u0}
U1 = {'type': 'ineq',
             'fun' : u1}
U2 = {'type': 'ineq',
             'fun' : u2}
U3 = {'type': 'ineq',
             'fun' : u3}
U4 = {'type': 'ineq',
             'fun' : u4}
U5 = {'type': 'ineq',
             'fun' : u5}
U6 = {'type': 'ineq',
             'fun' : u6}
U7 = {'type': 'ineq',
             'fun' : u7}
# x = minimize(objective, X0, method='trust-constr', options = {'disp':True}).x#, constraints = [ineq_cons], options={'ftol': 1e-9, 'disp': True})

In [84]:
import warnings
warnings.filterwarnings("ignore")

samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
results = []
for i in range(len(samples)):
    X0 = np.array(samples.iloc[i])
    res = minimize(objective, X0, method='COBYLA', 
               constraints = [ineq_cons, L0, L1, L2, L3, L4, L5, L6, L7, U0, U1, U2, U3, U4, U5, U6, U7], 
               options = {'verbose':1, 'display':1, 'maxiter':10}) # constraints = [constraint], options = {'verbose':1})#, bounds = bounds)
    results.append((X0, res.x, -res.fun))

In [85]:
sorted(results, key = lambda x: x[2], reverse = True)[:10]

[(array([3.84330328, 3.11903088, 8.63367165, 8.88974916, 8.81972552,
         3.69938965, 8.75334237, 8.74292895]),
  array([3.75545987, 2.17043145, 9.87274423, 8.8384022 , 9.82653983,
         3.52100479, 9.77981865, 9.75197606]),
  102075.54725547766),
 (array([2.32452262, 2.58871554, 9.32749721, 9.99315489, 4.52452962,
         7.02169385, 4.2840132 , 2.97129558]),
  array([ 2.        ,  2.        , 10.        ,  9.78758507,  4.42929722,
          6.43681893,  5.50151296,  3.9827708 ]),
  97462.75542251446),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.85636718, 2.        , 7.84065985, 7.09442702, 5.40725822,
         3.5565845 , 5.65175583, 3.72085619]),
  97361.97930286969),
 (array([8.76254164, 2.40324077, 8.77518014, 3.82322874, 4.85104019,
         4.60577037, 8.11653639, 8.55373399]),
  array([ 8.1369557 ,  2.        , 10.        ,  3.77095472,  4.85471749,
          5.18932401,  9.34236309,  8.5

# Alternative Search

In [88]:
def coordinateAscentQuad(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    M = {}
    c = 9
    for i in range(8):
        for j in range(i, 8):
            M[(i,j)] = c
            c += 1
    def partials(x):
        grad = []
        coef = alt_predictor.coef_
        for i in range(8):
            g = coef[1 + i]
            for j in range(8):
                if i == j:
                    g += 2*x[i]*coef[M[i,i]]
                else:
                    g += coef[M[min([i,j]), max([i,j])]]*x[j]
            grad.append(g)
        return np.array(grad)
    
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2) and qr.predict(poly.fit_transform([y])) > old: #((10 >= y[0] >= 6.0 and 5 >= y[1] >= 2 and 8 >= y[2] >= 3.5 and 4.0 >= y[3] >= 2.0) and (11 >= y[4] > 4 and 7 > y[5] > 3.5 and 8 > y[6] > 4 and 7 > y[7] > 2.5)) and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier(torch.from_numpy(poly.fit_transform([y])))[0]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [89]:
results = []
samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
for i in range(len(samples)):
    X0 = np.array(samples.iloc[i])
    result = coordinateAscentQuad(classif, ridge, X0, confidence = .6)
    print(X0, ridge.predict(poly.fit_transform([X0])), classif(torch.from_numpy(poly.fit_transform([X0]))))
    print(ridge.predict(poly.fit_transform([result])), result, classif(torch.from_numpy(poly.fit_transform([result]))))
    results.append((X0, result, ridge.predict(poly.fit_transform([result])), ridge.predict(poly.fit_transform([X0]))))

No more feasible directions at iteration 1875 . [26382.75]
[7.74386775 4.06369632 7.97899711 9.64484403 8.47690503 7.27085673
 4.46140229 6.17626983] [58815.74838186] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[83609.39980815] [7.5577634  2.00026277 8.39165309 9.56958246 8.43403031 6.93506399
 4.57966638 6.18334785] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 182 . [2933.]
[7.06856866 2.21573269 6.34825617 7.88644084 7.55687063 9.58805763
 3.48553571 3.46890728] [71884.60703795] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[74597.80185381] [7.04800105 2.00092853 6.39187744 7.88150927 7.54899506 9.56380815
 3.5006234  3.47009681] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2164 . [33209.25]
[5.5656446  4.48493532 9.23656697 7.38481998 7.2770088  6.54463189
 2.93160599 9.79820983] [60089.75505921] tensor([[1.0000

No more feasible directions at iteration 1098 . [16946.25]
[6.27364679 3.25691991 5.53904012 4.37568764 6.32528265 7.43092455
 3.56693159 9.59640105] [62792.35400913] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[78557.44929311] [6.15324117 2.0002209  5.84465548 4.36734079 6.31173851 7.25869161
 3.64875556 9.57538051] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1566 . [22629.5]
[2.12717718 4.24935594 4.83282894 9.48940368 6.69310519 4.95351523
 4.76269905 3.49531252] [57273.48107284] tensor([[0.9994]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[77921.15288727] [2.00003738 2.55510722 5.33879829 9.39859211 6.66004585 4.71934416
 4.80233832 3.50660002] tensor([[0.9958]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 3972 . [33281.]
[7.95462855 5.20594564 2.04653061 7.76957186 8.74530383 7.54793829
 5.16137886 7.01085736] [37898.16037398] tensor([[1.000

No more feasible directions at iteration 1961 . [26365.]
[7.83616152 4.12446818 8.26146515 7.8197602  2.56544452 9.52279921
 6.45425932 4.03578958] [59397.24175063] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[84005.11141747] [7.63101305 2.00013598 8.6365725  7.76443017 2.49028035 9.30122896
 6.56777954 4.01569402] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 5348 . [47300.75]
[8.04243762 6.54567728 9.72284694 2.59465914 4.73874635 8.58703844
 3.40612364 8.84626268] [42520.31523618] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[88326.25075242] [ 7.4853625   2.00024338 10.45196365  2.620206    4.66244171  7.70170846
  3.82171086  8.68409201] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2258 . [31125.75]
[7.10731359 4.39252568 6.82182993 3.48153784 3.91557955 5.82419792
 2.60837853 4.97694776] [57826.12845189] tenso

No more feasible directions at iteration 1609 . [22717.25]
[2.77580922 3.72582298 2.45069058 8.23582683 9.28100114 6.83650414
 4.17394074 6.97840703] [52127.04738712] tensor([[0.9998]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[73082.50716534] [2.64402005 2.00016076 3.00289291 8.16919477 9.25402261 6.65222449
 4.24539455 6.98094141] tensor([[0.9925]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1 . [0.75]
[5.25904766 8.20173937 7.41872267 8.21652658 2.64033946 8.93704144
 9.65310424 5.90079695] [37464.39119935] tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[37464.920639] [5.25903557 8.20153937 7.41882135 8.21648967 2.6403218  8.937045
 9.65308208 5.90076732] tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2164 . [33209.25]
[5.5656446  4.48493532 9.23656697 7.38481998 7.2770088  6.54463189
 2.93160599 9.79820983] [60089.75505921] tensor([[1.0000]], dty

No more feasible directions at iteration 313 . [329.5]
[3.35861851 8.00191927 9.67182006 7.29808488 9.83597288 6.28172953
 2.87220783 4.36908157] [41710.29764203] tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[42094.86808804] [3.34668313 7.90735773 9.70084855 7.28561002 9.83147491 6.23858798
 2.88020454 4.36691632] tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 3847 . [37961.]
[4.98997542 5.41873414 3.61500436 6.05701488 8.23160727 7.29218542
 3.97855472 9.33913974] [41508.4830794] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[77242.42510285] [4.70086684 2.00112981 4.68130943 5.98009335 8.1953207  6.84713329
 4.16203674 9.2810231 ] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
Non greedy step
[8.76254164 2.40324077 8.77518014 3.82322874 4.85104019 4.60577037
 8.11653639 8.55373399] [86914.966021] tensor([[0.0250]], dtype=torch.float64, grad_fn=<SigmoidBackwar

No more feasible directions at iteration 622 . [8816.75]
[9.49851485 2.66765712 2.51020965 4.38218088 4.97826409 7.07132962
 5.61656206 6.3882133 ] [59438.08887181] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[67390.51896883] [9.44390576 2.00029418 2.7111495  4.38386216 4.96241146 7.00514648
 5.64090224 6.38223437] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1 . [0.25]
[2.41785327 8.26680725 4.55367652 4.91451848 5.77937036 9.39042502
 2.42309088 4.50039664] [35336.2246027] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[35336.42206806] [2.41784573 8.26674864 4.55379519 4.91450455 5.77934201 9.39042384
 2.42310625 4.50036603] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[2.27568762 5.53307114 2.17336102 5.82804661 9.9896783  9.04718562
 9.71145997 9.88106918] [38480.9356788] tensor([[0.9998]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[61984.452841

No more feasible directions at iteration 777 . [11845.]
[7.64918917 2.89758583 7.32706996 4.79547298 5.36945525 9.89016881
 4.4248102  5.38032035] [67514.34161202] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[78582.22601178] [7.55430584 2.00024405 7.49528028 4.78914897 5.34190248 9.78799158
 4.49135415 5.37340151] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1497 . [21464.5]
[8.60608982 3.68830836 7.21414797 6.69336848 4.97587656 9.16413346
 8.72876187 7.56794622] [61238.15230665] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[81611.14458295] [8.44480216 2.0004751  7.55766568 6.66110664 4.94377909 8.99970533
 8.80486524 7.55359697] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1106 . [16311.25]
[9.22545751 3.26607377 8.6440655  9.16676316 4.04581057 9.74295102
 4.85377851 8.01883358] [64219.93497809] tensor([[1.0000

No more feasible directions at iteration 2043 . [29207.75]
[3.071609   4.24563298 5.1245792  7.19735674 6.06970858 7.5685513
 4.64635842 9.32825589] [54983.59460925] tensor([[0.9990]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[82330.37288984] [2.87615919 2.00016448 5.71248047 7.12618587 6.048156   7.30955181
 4.76427108 9.29211076] tensor([[0.8395]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[4.20966833 4.82755058 8.33112372 6.05386772 6.45425962 7.42645223
 9.66165679 9.32013025] [58803.94264895] tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[73073.45767184] [4.0835274  3.55766018 8.62038701 6.00814834 6.4537722  7.26956812
 9.70461783 9.3059548 ] tensor([[0.6007]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2459 . [30459.5]
[7.86588373 4.44265088 3.64158435 8.98533422 5.7447965  5.47726131
 7.45525759 3.07284886] [48959.30667726] tensor([[0.9994]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[765

No more feasible directions at iteration 659 . [11627.5]
[7.69025148 2.81017694 7.48039875 7.10897734 7.33196814 6.50889107
 4.53468652 6.53750297] [73055.49401906] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[83839.93886477] [7.61472645 2.00051189 7.65279317 7.09161085 7.31891098 6.3814551
 4.57885283 6.5377389 ] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 4719 . [19304.]
[2.11829325 6.82476744 2.0357269  9.60730567 3.60323836 8.49364242
 6.92712904 3.5485901 ] [32835.25559196] tensor([[0.9998]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[50815.44729406] [2.0000387  4.27432686 3.21081951 9.37976132 3.45460939 8.53811302
 6.88542586 3.48032127] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2 . [0.75]
[4.91825354 8.84363622 8.66625149 2.02358779 9.74090145 9.86560545
 7.37689289 9.83481264] [36816.27657038] tensor([[1.0000]], dtyp

No more feasible directions at iteration 3972 . [33281.]
[7.95462855 5.20594564 2.04653061 7.76957186 8.74530383 7.54793829
 5.16137886 7.01085736] [37898.16037398] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[68797.90852483] [7.73132528 2.00058436 3.16339209 7.68962515 8.66086158 7.21704695
 5.27232236 6.99697346] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 4307 . [40181.75]
[7.91940831 5.75010687 4.99891093 8.70989058 9.0463968  6.72622391
 6.36021882 9.53906911] [41795.74468662] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[80196.6212996] [7.63089449 2.00126525 6.07237853 8.57477028 9.02016847 6.16903961
 6.49094386 9.50953263] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[6.412598   8.27427726 8.51503918 8.93291554 4.91323368 8.51247272
 2.84247125 8.45886467] [36565.23176888] tensor([[1.0000]], dtyp

In [ ]:
# sorted(results, key = lambda x: x[2], reverse = True)

In [90]:
sorted(df['Altitude'], reverse = True)

[87504.62,
 86722.18,
 85846.05,
 85234.52,
 83528.12,
 83442.09,
 83051.34,
 82927.4,
 82061.53,
 81642.69,
 81555.23,
 81538.36,
 80338.0,
 79837.12,
 79274.52,
 78784.51,
 78464.02,
 78285.78,
 78227.84,
 77779.3,
 77117.16,
 76251.66,
 75869.35,
 74482.89,
 74264.98,
 74121.92,
 74057.05,
 73939.99,
 73631.16,
 73446.17,
 73263.34,
 72485.31,
 71832.66,
 71412.88,
 70886.62,
 70830.75,
 68788.69,
 68580.09,
 68412.84,
 67842.77,
 67582.01,
 67541.22,
 65967.9,
 65755.58,
 65196.87,
 64191.2,
 63832.05,
 63545.63,
 63428.37,
 62995.52,
 62259.07,
 61916.03,
 61777.55,
 60676.16,
 60241.04,
 59576.45,
 59528.58,
 59376.38,
 59242.72,
 59079.92,
 58941.78,
 58933.1,
 58280.82,
 58009.89,
 57964.61,
 57829.19,
 57613.95,
 57492.59,
 57476.48,
 57130.39,
 56982.9,
 56945.66,
 56933.59,
 56623.36,
 56583.86,
 56483.61,
 56466.14,
 55940.92,
 55933.92,
 55512.09,
 55372.77,
 55176.56,
 55084.65,
 54923.63,
 54917.07,
 54591.75,
 54481.37,
 54169.0,
 54082.5,
 53435.33,
 53361.03,
 53131.5

In [149]:
best = [3.79774661, 2.50143278, 8.75329661, 8.87230654, 8.79869793,
         3.59618521, 8.76162157, 8.74398984]

In [152]:
classif(torch.from_numpy(poly.fit_transform([best])))

tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)

In [192]:
# Interaction terms help marginally. 
#X_train, X_test = df.iloc[100:300, :-15], df.iloc[300:, :-15]
#X_train, X_test = df.iloc[100:, :-15], df.iloc[:100, :-15]
idxs = np.array(list(range(200, 400)) + list(range(100)))
X_train, X_test = poly.fit_transform(df.iloc[idxs, :-15]), poly.fit_transform(df.iloc[100:200, :-15])
#X_train, X_test = df.iloc[100:300, :-15], df.iloc[300:, :-15]

# Stability level 1: cluster into flight time of 20 and 90.
#Y_train, Y_test = df.iloc[:300, -1] > 90, df.iloc[300:, -1] > 90
#Y_train, Y_test = df.iloc[100:, -1] > 90, df.iloc[:100, -1] > 90
Y_train, Y_test = df.iloc[idxs, -1] > 90, df.iloc[100:200, -1] > 90
#Y_train, Y_test = df.iloc[:300, -1] > 90, df.iloc[300:, -1] > 90

reg = LR(penalty = 'l2')
reg.fit(X_train, Y_train)
sum(reg.predict(X_test) == Y_test)

93

In [162]:
poly

PolynomialFeatures()

Schord
Sspan
Ssweep
Stip
Bchord
Bspan
Bsweep
Btip
